## Imports

In [139]:
from numpy.random import seed
seed(888)
from tensorflow import random
random.set_seed(404)

In [140]:
import os
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from PIL import Image

from time import strftime

## Constants

In [141]:
X_TRAIN_PATH = "MNIST/digit_xtrain.csv"
X_TEST_PATH = "MNIST/digit_xtest.csv"
Y_TRAIN_PATH = "MNIST/digit_ytrain.csv"
Y_TEST_PATH = "MNIST/digit_ytest.csv"

LOGGING_PATH = "tensorboard_mnist_digit_logs/"

NR_CLASSES = 10
VALIDATION_SIZE = 1000
TOTAL_INPUTS = 784

## Get the Data

In [142]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=",", dtype = int)
y_train_all.shape

(60000,)

In [143]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=",", dtype = int)
y_test.shape

(10000,)

In [144]:
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=",", dtype = int)
x_train_all.shape

(60000, 784)

In [145]:
x_test = np.loadtxt(X_TEST_PATH, delimiter=",", dtype = int)
x_test.shape

(10000, 784)

## Data Preprocessing

In [146]:
# Re-scale
x_train_all , x_test = x_train_all/255.0 , x_test/255.0

#### Convert target to one-hot encoding

In [147]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_train_all.shape

(60000, 10)

In [148]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

#### Create validation dataset from training data

In [149]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [150]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [151]:
x_train.shape

(59000, 784)

In [152]:
x_val.shape

(1000, 784)

## Setup Tensorflow Graph

In [153]:
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder(tf.float32, shape= [None, TOTAL_INPUTS], name="X")
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, 10], name="labels")

#### Neural Network Architecture

In [154]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [155]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return layer_out

In [156]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [157]:
# initial_wl = tf.compat.v1.truncated_normal(shape= [TOTAL_INPUTS, n_hidden1], stddev= 0.1, seed=42)
# w1= tf.Variable(initial_value=initial_wl)

# initial_b1 = tf.constant( value= 0.0, shape= [n_hidden1])
# b1 = tf.Variable(initial_value = initial_b1)

# layer1_in = tf.matmul(X, w1) + b1

# layer1_out = tf.nn.relu(layer1_in)

In [158]:
# initial_w2 = tf.compat.v1.truncated_normal(shape=[n_hidden1, n_hidden2], stddev= 0.1, seed=42)
# w2 = tf.Variable(initial_value=initial_w2)

# initial_b2= tf.constant(value = 0.0, shape=[n_hidden2])
# b2 = tf.Variable(initial_value= initial_b2)

# layer2_in = tf.matmul(layer1_out, w2) + b2

# layer2_out = tf.nn.relu(layer2_in)

In [159]:
# initial_w3 = tf.compat.v1.truncated_normal(shape=[n_hidden2, 10], stddev= 0.1, seed=42)
# w3 = tf.Variable(initial_value=initial_w3)

# initial_b3= tf.constant(value = 0.0, shape=[10])
# b3 = tf.Variable(initial_value= initial_b3)

# layer3_in = tf.matmul(layer2_out, w3) + b3

# output = tf.nn.softmax(layer3_in)

## Tensorboard Setup

In [160]:
folder_name=f'Model 1 at{strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print("Successfully created directories")

Successfully created directories


## Loss, Optimisation & Matrix

#### Defining Loss Func

In [161]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining Optimizer

In [162]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [163]:
with tf.name_scope('accuracy_calc'):
    model_prediction = tf.argmax(output, axis=1, name='prediction')
    correct_pred = tf.equal(model_prediction, tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [164]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

In [165]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

## Run Session

In [166]:
sess = tf.compat.v1.Session()

#### Setup Filewriter and Merger

In [167]:
merged_summary = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + "/validation")

In [168]:
init = tf.global_variables_initializer()
sess.run(init)

## Batching the Data

In [169]:
size_of_batch = 1000

In [170]:
nr_examples= y_train.shape[0]
nr_iterations = int(nr_examples/size_of_batch)

index_in_epoch = 0

In [171]:
def next_batch(batch_size, data, labels):

    global nr_examples
    global index_in_epoch

    start = index_in_epoch
    index_in_epoch = index_in_epoch + batch_size

    if index_in_epoch > nr_examples:
        start = 0
        index_in_epoch = batch_size

    end = index_in_epoch

    return data[start:end], labels[start:end]


## Training Loop

In [173]:
for epoch in range(nr_epochs):
    for i in range(nr_iterations):

        # =============Training Dataset=========

        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
        
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')

    	# ===============Validation===========

    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary,epoch)

print("# To Train Them İs My Cause!!!.. #")

Epoch 0 	| Training Accuracy = 0.8619999885559082
Epoch 1 	| Training Accuracy = 0.8859999775886536
Epoch 2 	| Training Accuracy = 0.8849999904632568
Epoch 3 	| Training Accuracy = 0.925000011920929
Epoch 4 	| Training Accuracy = 0.9760000109672546
Epoch 5 	| Training Accuracy = 0.9769999980926514
Epoch 6 	| Training Accuracy = 0.9760000109672546
Epoch 7 	| Training Accuracy = 0.9829999804496765
Epoch 8 	| Training Accuracy = 0.9789999723434448
Epoch 9 	| Training Accuracy = 0.9850000143051147
Epoch 10 	| Training Accuracy = 0.9890000224113464
Epoch 11 	| Training Accuracy = 0.9869999885559082
Epoch 12 	| Training Accuracy = 0.9869999885559082
Epoch 13 	| Training Accuracy = 0.9860000014305115
Epoch 14 	| Training Accuracy = 0.9879999756813049
Epoch 15 	| Training Accuracy = 0.9879999756813049
Epoch 16 	| Training Accuracy = 0.9890000224113464
Epoch 17 	| Training Accuracy = 0.9879999756813049
Epoch 18 	| Training Accuracy = 0.9890000224113464
Epoch 19 	| Training Accuracy = 0.98900002

## Saving the model

In [175]:
outputs = {"accuracy_calc/prediction": model_prediction}
inputs = {'X': X}
tf.saved_model.simple_save(sess,"SavedModel", inputs, outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel\saved_model.pb


## Make a Prediction

In [176]:
img = Image.open("MNIST/test_img.png")

In [177]:
bw = img.convert("L")
img_array = np.invert(bw)

In [178]:
test_img = img_array.ravel()
test_img.shape

(784,)

In [179]:
pred = sess.run(fetches=tf.argmax(output, axis=1),
 feed_dict={X:[test_img]})

In [180]:
print(f"Prediction for test image is {pred}")

Prediction for test image is [2]


## Test and Evaluate

In [181]:
test_acc=sess.run(fetches= accuracy, feed_dict={X:x_test, Y: y_test})
print(f"Accuracy on test set is {test_acc:0.2%}")

Accuracy on test set is 97.89%


## Reset for Next Run

In [182]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()